<a href="https://colab.research.google.com/github/boldasl/20_IT_SYSTEM_DESIGN_008/blob/master/HW3_AugmentedMNIST_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing.image import *

print("# Tensorflow Version:", tf.__version__)

# Tensorflow Version: 2.2.0


Using TensorFlow backend.


In [0]:
# data augmentation config
RotationRange = 30
ZoomRange = (1.3, 1.3)
FlipRate = 0.3

SavePath = "./drive/My Drive/Colab Notebooks/param/Augmented_MNIST_Keras/weight.h5"
Mode = "CNN"
TrainFlag = True


In [4]:
# load mnist dataset
((train_images, train_labels), (test_images, test_labels)) = mnist.load_data()

train_images = train_images.reshape(-1, 28, 28, 1) / 255.0 # expand channel axis & normalize
test_images = test_images.reshape(-1, 28, 28, 1) / 255.0
train_labels = to_categorical(train_labels) # one-hot encoding
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 2s 0us/step


In [5]:
# data augmentation
_train_images = np.reshape(train_images, [-1, 28, 28, 1]) # reshape mnist to 3d array including channel axis
train_images_aug = []
_test_images = np.reshape(test_images, [-1, 28, 28, 1]) # reshape mnist to 3d array including channel axis
test_images_aug = []

# data augmentation
# training data augmentation
for idx in range(train_images.shape[0]): 
    temp = random_rotation(_train_images[idx], rg=RotationRange, row_axis=0, col_axis=1, channel_axis=2)
    temp = random_zoom(temp, zoom_range=ZoomRange, row_axis=0, col_axis=1, channel_axis=2)
    if np.random.rand() < FlipRate: temp = np.flip(temp, axis=1)
    train_images_aug.append(temp)
    if idx % 100 == 0: print("Train Image Augmentation Processing... {0:0.1f}%\r".format(idx/train_images.shape[0]*100), end='')
print("Train Image Augmentation Complete!                         ")

# test data augmentation
for idx in range(test_images.shape[0]):
    temp = random_rotation(_test_images[idx], rg=RotationRange, row_axis=0, col_axis=1, channel_axis=2)
    temp = random_zoom(temp, zoom_range=ZoomRange, row_axis=0, col_axis=1, channel_axis=2)
    if np.random.rand() < FlipRate: temp = np.flip(temp, axis=1)
    test_images_aug.append(temp)
    if idx % 100 == 0: print("Test Image Augmentation Processing... {0:0.1f}%\r".format(idx/train_images.shape[0]*100), end='')
print("Test Image Augmentation Complete!                         ")

# convert to numpy array
train_images_aug = np.array(train_images_aug)
test_images_aug = np.array(test_images_aug)

Train Image Augmentation Complete!                         
Test Image Augmentation Complete!                         


In [0]:
def CNN(summary=True):
    model = Sequential()
    temp = Conv2D(32, (5, 5, ), activation="relu", input_shape=(28, 28, 1), padding="same")
    print(type(temp))
    model.add(temp)
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (5, 5, ), activation="relu", padding="same"))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10,activation="softmax"))
    if summary: print(model.summary())

    return model

def FC(summary=True):
    model = Sequential()
    model.add(layers.Dense(784, activation="relu", input_shape=(784,)))
    model.add(layers.Dense(392, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    if summary: print(model.summary())
    return model

In [7]:
if Mode is "CNN":
    train_images = train_images.reshape(-1, 28, 28, 1)
    test_images = test_images.reshape(-1, 28, 28, 1)
    train_images_aug = train_images_aug.reshape(-1, 28, 28, 1)
    test_images_aug = test_images_aug.reshape(-1, 28, 28, 1)

    nn = CNN(summary=True)

elif Mode is "FC":
    train_images = train_images.reshape(-1, 784)
    test_images = test_images.reshape(-1, 784)
    train_images_aug = train_images_aug.reshape(-1, 784)
    test_images_aug = test_images_aug.reshape(-1, 784)

    nn = FC(summary=True)
nn.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) 

if TrainFlag:
    for _ in range(3):
        nn.fit(train_images,train_labels, epochs=1, batch_size=100)
        nn.fit(train_images_aug,train_labels, epochs=1, batch_size=100)
    nn.save_weights(SavePath)
else: nn.load_weights(SavePath)


_, acc_orig = nn.evaluate(test_images,test_labels)
_, acc_aug = nn.evaluate(test_images_aug, test_labels)

print("\nAccuracy")
print("Original : {0:0.4f}".format(acc_orig))
print("Augmented: {0:0.4f}".format(acc_aug))

<class 'keras.layers.convolutional.Conv2D'>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dense_2 (D